In [1]:
import pandas as pd
from pandas import DataFrame
df_tennis = pd.read_csv('PlayTennis.csv')
print("\n Given Play Tennis Data Set:\n\n",df_tennis)


 Given Play Tennis Data Set:

    PlayTennis   Outlook Temperature Humidity    Wind
0          No     Sunny         Hot     High    Weak
1          No     Sunny         Hot     High  Strong
2         Yes  Overcast         Hot     High    Weak
3         Yes      Rain        Mild     High    Weak
4         Yes      Rain        Cool   Normal    Weak
5          No      Rain        Cool   Normal  Strong
6         Yes  Overcast        Cool   Normal  Strong
7          No     Sunny        Mild     High    Weak
8         Yes     Sunny        Cool   Normal    Weak
9         Yes      Rain        Mild   Normal    Weak
10        Yes     Sunny        Mild   Normal  Strong
11        Yes  Overcast        Mild     High  Strong
12        Yes  Overcast         Hot   Normal    Weak
13         No      Rain        Mild     High  Strong


In [3]:
df_tennis.columns[0]
df_tennis.keys()[0]

'PlayTennis'

In [4]:
def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2)for prob in probs])

In [5]:
def entropy_of_list(a_list):
    from collections import Counter
    cnt=Counter(x for x in a_list)
    num_instances=len(a_list)*1.0
    print("\n Number of instances of current Sub Class is {0}:".format(num_instances)) 
    probs=[x/num_instances for x in cnt.values()]
    print("\n Classes:",min(cnt),max(cnt))
    print("\n Probabilities of Class {0} is {1}:".format (min(cnt),min(probs)))
    print("\n probabilities of Class {0} is {1}:".format (max(cnt),max(probs)))
    return entropy(probs)

In [6]:
print("\n INPUT DATA SET FOR ENTROPY CALCULATION:\n",df_tennis['PlayTennis'])
total_entropy = entropy_of_list(df_tennis['PlayTennis'])
print("\n Total Entropy of Play Tennis Data Set:",total_entropy)


 INPUT DATA SET FOR ENTROPY CALCULATION:
 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: PlayTennis, dtype: object

 Number of instances of current Sub Class is 14.0:

 Classes: No Yes

 Probabilities of Class No is 0.35714285714285715:

 probabilities of Class Yes is 0.6428571428571429:

 Total Entropy of Play Tennis Data Set: 0.9402859586706309


In [7]:
def information_gain(df,split_attribute_name,target_attribute_name,trace=0):
    print("information_Gain Calculation of",split_attribute_name)
    df_split=df.groupby(split_attribute_name)
    nobs=len(df.index)*1.0
    print("NOBS",nobs)
    df_agg_ent=df_split.agg({target_attribute_name:[entropy_of_list,lambda x:len(x)/nobs]})[target_attribute_name]
    df_agg_ent.columns=['Entropy','PropObservations']
    new_entropy=sum(df_agg_ent['Entropy']*df_agg_ent['PropObservations'])
    old_entropy=entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy

In [8]:
print('Info_gain for Outlook is:'+str(information_gain(df_tennis,'Outlook','PlayTennis')),"\n")
print("\n Info_gain for Humidity is:"+str(information_gain(df_tennis,'Humidity','PlayTennis')),"\n")
print("\n Info_gain for Wind is:"+str(information_gain(df_tennis,'Wind','PlayTennis')),"\n")
print("\n Info_gain for Temperature is:" +str(information_gain(df_tennis,'Temperature','PlayTennis')),"\n")

information_Gain Calculation of Outlook
NOBS 14.0

 Number of instances of current Sub Class is 4.0:

 Classes: Yes Yes

 Probabilities of Class Yes is 1.0:

 probabilities of Class Yes is 1.0:

 Number of instances of current Sub Class is 5.0:

 Classes: No Yes

 Probabilities of Class No is 0.4:

 probabilities of Class Yes is 0.6:

 Number of instances of current Sub Class is 5.0:

 Classes: No Yes

 Probabilities of Class No is 0.4:

 probabilities of Class Yes is 0.6:

 Number of instances of current Sub Class is 14.0:

 Classes: No Yes

 Probabilities of Class No is 0.35714285714285715:

 probabilities of Class Yes is 0.6428571428571429:
Info_gain for Outlook is:0.2467498197744391 

information_Gain Calculation of Humidity
NOBS 14.0

 Number of instances of current Sub Class is 7.0:

 Classes: No Yes

 Probabilities of Class No is 0.42857142857142855:

 probabilities of Class Yes is 0.5714285714285714:

 Number of instances of current Sub Class is 7.0:

 Classes: No Yes

 Probabi

In [9]:
def id3(df, target_attribute_name, attribute_names, default_class=None):
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt) == 1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class 
    else:
        default_class = max(cnt.keys()) 
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
        index_of_max = gainz.index(max(gainz)) 
        best_attr = attribute_names[index_of_max]
        tree = {best_attr:{}} 
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
    for attr_val, data_subset in df.groupby(best_attr):
        subtree = id3(data_subset,target_attribute_name,remaining_attribute_names, default_class)
        tree[best_attr][attr_val] = subtree
    return tree

 

In [10]:
attribute_names=list(df_tennis.columns)
print("List of Attributes:",attribute_names)
attribute_names.remove('PlayTennis')
print("Predicting Attributes:",attribute_names)

List of Attributes: ['PlayTennis', 'Outlook', 'Temperature', 'Humidity', 'Wind']
Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [11]:
from pprint import pprint
tree = id3(df_tennis , 'PlayTennis', attribute_names)
print("\n\n The Resultant Decision Tree is:\n")
pprint(tree)
attribute=next(iter(tree)) 
print("Best Attribute:\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

information_Gain Calculation of Outlook
NOBS 14.0

 Number of instances of current Sub Class is 4.0:

 Classes: Yes Yes

 Probabilities of Class Yes is 1.0:

 probabilities of Class Yes is 1.0:

 Number of instances of current Sub Class is 5.0:

 Classes: No Yes

 Probabilities of Class No is 0.4:

 probabilities of Class Yes is 0.6:

 Number of instances of current Sub Class is 5.0:

 Classes: No Yes

 Probabilities of Class No is 0.4:

 probabilities of Class Yes is 0.6:

 Number of instances of current Sub Class is 14.0:

 Classes: No Yes

 Probabilities of Class No is 0.35714285714285715:

 probabilities of Class Yes is 0.6428571428571429:
information_Gain Calculation of Temperature
NOBS 14.0

 Number of instances of current Sub Class is 4.0:

 Classes: No Yes

 Probabilities of Class No is 0.25:

 probabilities of Class Yes is 0.75:

 Number of instances of current Sub Class is 4.0:

 Classes: No Yes

 Probabilities of Class No is 0.5:

 probabilities of Class Yes is 0.5:

 Number

In [12]:
def classify(instance, tree, default=None): 
    attribute =next(iter(tree))
    if instance[attribute] in tree[attribute].keys(): 
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict): 
            return classify(instance, result)
        else:
            return result
    else:
        return default

In [13]:
df_tennis['predicted'] = df_tennis.apply(classify, axis=1, args=(tree,'No') )
print('Accuracy is:' + str( sum(df_tennis['PlayTennis']==df_tennis['predicted'] ) /(1.0*len(df_tennis.index)) ))
df_tennis[['PlayTennis', 'predicted']]

Accuracy is:1.0


,PlayTennis,predicted
0,No,No
1,No,No
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,No,No
6,Yes,Yes
7,No,No
8,Yes,Yes
9,Yes,Yes


In [14]:
training_data = df_tennis.iloc[1:4]
test_data = df_tennis.iloc[-4:]
train_tree = id3(training_data, 'PlayTennis', attribute_names)
test_data['predicted2'] = test_data.apply(classify, axis=1, args=(train_tree, 'Yes'))
print('\n\n Accuracy is : ' + str(sum(test_data['PlayTennis'] == test_data['predicted2'])/(1.0*len(test_data.index))))

information_Gain Calculation of Outlook
NOBS 3.0

 Number of instances of current Sub Class is 1.0:

 Classes: Yes Yes

 Probabilities of Class Yes is 1.0:

 probabilities of Class Yes is 1.0:

 Number of instances of current Sub Class is 1.0:

 Classes: Yes Yes

 Probabilities of Class Yes is 1.0:

 probabilities of Class Yes is 1.0:

 Number of instances of current Sub Class is 1.0:

 Classes: No No

 Probabilities of Class No is 1.0:

 probabilities of Class No is 1.0:

 Number of instances of current Sub Class is 3.0:

 Classes: No Yes

 Probabilities of Class No is 0.3333333333333333:

 probabilities of Class Yes is 0.6666666666666666:
information_Gain Calculation of Temperature
NOBS 3.0

 Number of instances of current Sub Class is 2.0:

 Classes: No Yes

 Probabilities of Class No is 0.5:

 probabilities of Class Yes is 0.5:

 Number of instances of current Sub Class is 1.0:

 Classes: Yes Yes

 Probabilities of Class Yes is 1.0:

 probabilities of Class Yes is 1.0:

 Number of 

<ipython-input-14-6443b9ba4d19>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted2'] = test_data.apply(classify, axis=1, args=(train_tree, 'Yes'))
